In [8]:
import numpy as np
import sys
# option to import from github folder
sys.path.insert(0, 'E:/Users/samsoon.inayat/T_Drive/GitHub/Downloaded/suite2p/')
# from suite2p import run_s2p
import suite2p

In [10]:
# set your options for running
ops = suite2p.default_ops() # populates ops with the default options
print(ops)

{'suite2p_version': '0.9.3', 'look_one_level_down': False, 'fast_disk': [], 'delete_bin': False, 'mesoscan': False, 'bruker': False, 'h5py': [], 'h5py_key': 'data', 'save_path0': [], 'save_folder': [], 'subfolders': [], 'move_bin': False, 'nplanes': 1, 'nchannels': 1, 'functional_chan': 1, 'tau': 1.0, 'fs': 10.0, 'force_sktiff': False, 'frames_include': -1, 'multiplane_parallel': False, 'preclassify': 0.0, 'save_mat': False, 'save_NWB': False, 'combined': True, 'aspect': 1.0, 'do_bidiphase': False, 'bidiphase': 0, 'bidi_corrected': False, 'do_registration': 1, 'two_step_registration': False, 'keep_movie_raw': False, 'nimg_init': 300, 'batch_size': 500, 'maxregshift': 0.1, 'align_by_chan': 1, 'reg_tif': False, 'reg_tif_chan2': False, 'subpixel': 10, 'smooth_sigma_time': 0, 'smooth_sigma': 1.15, 'th_badframes': 1.0, 'norm_frames': True, 'force_refImg': False, 'pad_fft': False, 'nonrigid': True, 'block_size': [128, 128], 'snr_thresh': 1.2, 'maxregshiftNR': 5, '1Preg': False, 'spatial_hp':

In [ ]:
# provide an h5 path in 'h5py' or a tiff path in 'data_path'
# db overwrites any ops (allows for experiment specific settings)
db = {
      'h5py': [], # a single h5 file path
      'h5py_key': 'data',
      'look_one_level_down': False, # whether to look in ALL subfolders when searching for tiffs
      'data_path': ['C:/Users/carse/github/tiffs'], # a list of folders with tiffs 
                                             # (or folder of folders with tiffs if look_one_level_down is True, or subfolders is not empty)
                                            
      'subfolders': [], # choose subfolders of 'data_path' to look in (optional)
      'fast_disk': 'C:/BIN', # string which specifies where the binary file will be stored (should be an SSD)
    }

# run one experiment
opsEnd=run_s2p.run_s2p(ops=ops,db=db)

In [ ]:
# make a list of db's and loop over them
db = []
db.append({'data_path': ['C:/Users/carse/github/tiffs']})
db.append({'data_path': ['C:/Users/carse/github/tiffs2']})

for dbi in db:
    opsEnd=run_s2p.run_s2p(ops=ops,db=dbi)

In [ ]:
# only run on specified tiffs
db = {
      'h5py': [], # a single h5 file path
      'h5py_key': 'data',
      'look_one_level_down': False, # whether to look in ALL subfolders when searching for tiffs
      'data_path': ['C:/Users/carse/github/tiffs/'], 
                    # a list of folders with tiffs 
                    # (or folder of folders with tiffs if look_one_level_down is True, or subfolders is not empty)        
      'subfolders': [], # choose subfolders of 'data_path' to look in (optional)
      'fast_disk': 'C:/BIN', # string which specifies where the binary file will be stored (should be an SSD)
      'tiff_list': ['file022.tif', 'file023.tif'] # list of tiffs in folder * data_path *!
    }


# run one experiment
opsEnd=run_s2p.run_s2p(ops=ops,db=db)


In [ ]:
# make a list of all files in your folder (starting with 73.)
### *** this is for files that don't end with *.tif or *.tiff ***

import glob, os
froot = '/media/carsen/DATA2/grive/suite2p_test/example-tiffs-metafluor/'
files = glob.glob(os.path.join(froot, "73.*"))
for n,f in enumerate(files):
    files[n] = os.path.basename(f)
    
db = {
      'h5py': [], # a single h5 file path
      'h5py_key': 'data',
      'look_one_level_down': False, # whether to look in ALL subfolders when searching for tiffs
      'data_path': [froot], 
                            # a list of folders with tiffs 
                            # (or folder of folders with tiffs if look_one_level_down is True, or subfolders is not empty)
                            
      'subfolders': [], # choose subfolders of 'data_path' to look in (optional)
      'tiff_list': files # list of tiffs in folder * data_path *!
    }


# run one experiment
opsEnd=run_s2p.run_s2p(ops=ops,db=db)


In [ ]:
## change the save directory from 'suite2p' to a chosen name
# note the fast_disk will always be in 'suite2p', just the save_path will change

ops = run_s2p.default_ops() # populates ops with the default options
ops['sparse_mode'] = 1
ops['threshold_scaling'] = 3.0
db = {
      'h5py': [], # a single h5 file path
      'h5py_key': 'data',
      'look_one_level_down': False, # whether to look in ALL subfolders when searching for tiffs
      'data_path': ['D:/DATA/GT1/singlechannel_half/'], # a list of folders with tiffs 
                # (or folder of folders with tiffs if look_one_level_down is True, or subfolders is not empty)
      'save_folder': 'suite2p_sparse_mode'
    }

# run one experiment
opsEnd=run_s2p.run_s2p(ops=ops,db=db)

In [ ]:
# h5py file with multiple data fields (untested)

ops = run_s2p.default_ops() # populates ops with the default options
ops['nplanes'] = 12
ops['nchannels'] = 2
ops['fs'] = 5.0

db = {
      'h5py': 'C:/Users/carse/H5/data1.h5', # a single h5 file 
      'h5py_key': ['data'], # list of keys to use (they will be extracted in the order you give them)
      'look_one_level_down': True, # for h5 files, whether to use all files in same folder
      'data_path': [] # keep this empty!
    }

# run one experiment
opsEnd=run_s2p.run_s2p(ops=ops,db=db)